<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2021/blob/main/EqualAreaHistogram_NonInteger_Edges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.io as pio
import pandas as pd
pio.templates.default='plotly_dark'

In [3]:
def equal_area_histogram(data, nbins=None):
    data_sorted = np.sort(data)
    n=data_sorted.size
    nbins = int(2*n**(2/5)) if nbins is None else nbins
    nperbin = int(n//nbins)
    x = np.arange(0,n)
    edges = np.linspace(0,n,nbins+1)
    offsets = np.arange(np.ceil(-nperbin/4),np.floor(nperbin/4))
    indices=(edges[1:-1,None] + offsets[None,:]).astype(np.int64)
    Y=np.take(data_sorted,indices)
    X=np.stack((indices,np.ones_like(indices)),axis=2)
    XT = np.moveaxis(X,(0,1,2), (0,2,1))
    pinv=np.linalg.inv(XT@X) @ XT 
    mb=np.einsum('ijk,ik->ij',pinv,Y)
    smoothed_edges = np.r_[(np.min(data), mb[:,0]*edges[1:-1] + mb[:,1], np.max(data))]
    height = 1/nbins/(smoothed_edges[1:]-smoothed_edges[:-1])
    bin_edges=np.repeat(smoothed_edges,2)
    bin_heights=np.r_[0.,np.repeat(height,2),0.]

    return bin_edges, bin_heights

In [4]:
data = np.random.normal(loc=0.,scale=1., size=int(1e4))
x,y=equal_area_histogram(data)

fig3=make_subplots()
fig3.add_scatter(x=x, y=y, mode='lines')
fig3.update_layout(width=800,height=600)